# Hillmaker - basic usage

In this notebook we'll focus on basic use of hillmaker for analyzing arrivals, departures, and occupancy by time of day and day of week for a typical *discrete entity flow system*. A few examples of such systems include:

- patients arriving, undergoing some sort of care process and departing some healthcare system (e.g. emergency department, surgical recovery, nursing unit, outpatient clinic, and many more)
- customers renting, using, and returning bikes in a bike share system,
- users of licensed software checking out, using, checking back in a software license,
- products undergoing some sort of manufacturing or assembly process - occupancy is WIP,
- patrons arriving, dining and leaving a restaurant,
- travelers renting, residing in, and checking out of a hotel,
- flights taking off and arriving at their destination,
- ...

Basically, any sort of discrete [stock and flow system](https://en.wikipedia.org/wiki/Stock_and_flow) for which you are interested in time of day and day of week specific statistical summaries of occupancy, arrivals and departures, and have raw data on the arrival and departure times, is fair game for hillmaker.

## A prototypical example of a hillmaker use case

Patients flow through a short stay unit for a variety of procedures, tests or therapies. Let's assume patients can be classified into one of five categories of patient types: ART (arterialgram), CAT (post cardiac-cath), MYE (myelogram), IVT (IV therapy), and OTH (other). From one of our hospital information systems we were able to get raw data about the entry and exit times of each patient and exported the data to a csv file. Let's take a peek at the data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

ssu_stopdata = './input/ShortStay.csv'
stops_df = pd.read_csv(ssu_stopdata, parse_dates=['InRoomTS','OutRoomTS'])
stops_df.info() # Check out the structure of the resulting DataFrame

In [ ]:
stops_df.head()

As part of an operational analysis we would like to compute a number of relevant statistics, such as:

- mean and 95th percentile of overall SSU occupancy by hour of day and day of week,
- similar hourly statistics for patient arrivals and departures,
- all of the above but by patient type as well.

In addition to tabular summaries, plots are needed. Like this:

![SSU occupancy plot](images/ssu-occ.png)

Hillmaker was designed for precisely this type of problem. In fact, the very first version of hillmaker was written for analyzing an SSU when the author was an undergraduate interning at a large health care system. That very first version was written in BASIC on a [DECwriter](https://en.wikipedia.org/wiki/DECwriter)!

![DECwriter](images/DECwriter,_Tektronix,_PDP-11_(192826605).jpg)
<p align = "center">
<font size="-2">Source: By Wolfgang Stief from Tittmoning, Germany - DECwriter, Tektronix, PDP-11, CC0, https://commons.wikimedia.org/w/index.php?curid=105322423</font>
</p>

Over the years, hillmaker was migrated to [FoxPro](https://en.wikipedia.org/wiki/FoxPro), and then to MS Access where it [lived for many years](http://hillmaker.sourceforge.net/). In 2016, I [moved it to Python](https://misken.github.io/blog/hillmaker-python-released/).

## Current (2022-06-28) status of code

Version 0.4.0 was just released and is available on [PyPI](https://pypi.org/project/hillmaker/) or source from https://github.com/misken/hillmaker. This version is much faster than previous versions (thank you numpy) and includes a CLI, flow conservation checks and better logging. It does however revert back to only allowing a single category field (multiple category fields can easily be handled by constructing composite category strings). You can read more about this latest release of Hillmaker at [https://misken.github.io/blog/hillmaker-python-released/](https://misken.github.io/blog/hillmaker-python-released/). It's free and open source.

Hillmaker is implemented as an importable Python module and as a runnable script with a simple CLI. This new version of Hillmaker is still in what I'd call a pre-release state. The output does match the Access version for the ShortStay database that I included in the original Hillmaker. I've been actively using it to process thousands of simulation output log files as part of a research project on OB patient flow. More testing is needed before I release it publicly as version 1.0, but it does appear to be doing its primary job correctly. Please open an issue on GitHub if you think it's computing something incorrectly. Before using for any real project work, you should do your own testing to confirm that it is working appropriately for your needs. Use at your own risk - see [LICENSE file in GitHub](https://github.com/misken/hillmaker/blob/master/LICENSE).

## User interface plans
Over the years, I (and many others) have used the old Hillmaker in a variety of ways, including:

- MS Access form based GUI
- run main Hillmaker sub from Access VBA Immediate Window
- run Hillmaker main sub (and/or components subs) via custom VBA procedures

I'd like users to be able to use the new Python based version in a number of different ways as well. As I'll show in this Jupyter notebook, it can be used by importing the `hillmaker` module and then calling Hillmaker functions via:

- a Jupyter notebook (or any Python terminal such as an IPython shell or QT console, or IDLE)
- a Python script with the input arguments set and passed via Python statements

### A CLI

While these two options provide tons of flexibility for power users, I have also added a CLI. The CLI is demo'd in this notebook as well.

### A GUI for hillmaker
This is uncharted territory for me. Python has [a number of frameworks/toolkits for creating GUI apps](https://wiki.python.org/moin/GuiProgramming). This is not the highest priority for me but I do plan on creating a GUI for Hillmaker. If anyone wants to help with this, let me know.



## Installing Hillmaker

Whereas the old Hillmaker required MS Access, the new one requires an installation of Python 3.7+ along with several Python modules that are widely used for analytics and data science work. The free and open source [Anaconda Distribution of Python](https://www.anaconda.com/products/distribution) is a great way to get started with Python for analytics work. It is available on all platforms. Once you've got a working version of Python, you can install into a Python (or Conda) virtual environment with:

```
pip install hillmaker
```

This latest version (0.4.0) should be available on `conda-forge` soon.

Obviously if you are comfortable working with source code, you can also install `hillmaker` from its GitHub repo or a clone/fork of it.

## Module imports
To run Hillmaker we only need to import a few modules. Since the main hillmaker function uses pandas DataFrames for both data input and output, we need to import `pandas` in addition to `hillmaker`.

In [ ]:
import pandas as pd
import hillmaker as hm

## Read main stop data file
Here's the first few lines from our csv file containing the patient stop data:

    PatID,InRoomTS,OutRoomTS,PatType
    1,1/1/1996 7:44,1/1/1996 8:50,IVT
    2,1/1/1996 8:28,1/1/1996 9:20,IVT
    3,1/1/1996 11:44,1/1/1996 13:30,MYE
    4,1/1/1996 11:51,1/1/1996 12:55,CAT
    5,1/1/1996 12:10,1/1/1996 13:00,IVT
    6,1/1/1996 14:16,1/1/1996 15:35,IVT
    7,1/1/1996 14:40,1/1/1996 15:25,IVT


We have already read this data into a pandas DataFrame named `stops_df`. Each record is a "stop" at the SSU.

Check out the top and bottom of `stops_df`. 

In [ ]:
stops_df.head(7)

In [ ]:
stops_df.tail(7)

Let's compute some basic summary statistics such as the earliest and latest arrival and departure as well as counts by patient type.

In [ ]:
print(f'Earliest arrival = {stops_df["InRoomTS"].min()}')
print(f'Latest departure = {stops_df["OutRoomTS"].max()}')

In [ ]:
stops_df.groupby(['PatType'])['PatID'].count()

Let's get a sense of the number of patient visits by month.

In [ ]:
stops_df['InRoomTS'].groupby(stops_df.InRoomTS.dt.to_period("M")).agg('count')

You probably want to do some length of stay analysis, so let's compute it in hours and then do `describe` by patient type.

In [ ]:
stops_df['LOS'] = (stops_df['OutRoomTS'] - stops_df['InRoomTS']) / pd.Timedelta(1, "h")

In [ ]:
stops_df.groupby(['PatType'])['LOS'].describe()

No obvious problems. We'll assume the data was all read in correctly.

## Creating occupancy summaries
The primary function in hillmaker is called `make_hills` and plays the same role as the `Hillmaker` function in the original Access VBA version of Hillmaker. Let's get a little help on this function.

In [ ]:
help(hm.make_hills)

Most of the parameters are similar to those in the original VBA version, though a few new ones have been added. For example, the `cat_to_exclude` parameter allows you to specify a list of category values for which you do not want occupancy statistics computed. Also, since the VBA version used an Access database as the container for its output, new parameters were added to control output to csv files instead.

### Example 1: 60 minute bins, all categories, export to csv
Specify values for all the required inputs:

In [ ]:
# Required inputs
scenario = 'ss_example_1'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start = '1/1/1996'
end = '9/30/1996 23:45'

# Optional inputs
verbose = 1 # INFO level logging


Now we'll call the main `make_hills` function. We won't capture the return values but will simply take the default behavior of having the summaries exported to csv files. You'll see that the filenames will contain the scenario value.

In [ ]:
hm.make_hills(scenario, stops_df, in_fld_name, out_fld_name, start, end, cat_fld_name, verbose=1)

Here's a screenshot of the current folder containing this IPython notebook (**basic_usage_shortstay_unit.ipynb**) and the csv files created by Hillmaker. 

![folder with output csv files](example_1_files.png)

If you've used the previous version of Hillmaker, you'll recognize these files. A few more statistics have been added, but otherwise they are the same. These csv files can be imported into a spreadsheet application for plot creation. Of course, we can also make plots in Python. We'll do that in the next example. 

![folder with output csv files](example_1_occ.png)

The files with 'cat' in their name are new. They contain summary overall summary statistics by category. In other words, they are NOT by time of day and day of week.

![folder with output csv files](example_1_occ_cat.png)

### Example 2: 30 minute bins, only CAT and IVT, return values to DataFrames

In [ ]:
# Required inputs - same as Example 1 except for scenario name
scenario = 'ss_example_2'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start = '1/1/1996'
end = '3/30/1996 23:45'

# Optional inputs
tot_fld_name = 'CAT_IVT' # Just to make it clear that it's only these patient types
bin_mins = 30 # Half-hour time bins
exclude = ['ART','MYE','OTH'] # Tell Hillmaker to ignore these patient types


Now we'll call `make_hills` and tuck the results (a dictionary of DataFrames) into a local variable. Then we can explore them a bit with Pandas.

In [ ]:
results_ex2 = hm.make_hills(scenario, stops_df, in_fld_name, out_fld_name, start, end, cat_fld_name,
                            total_str=tot_fld_name, bin_size_minutes=bin_mins, 
                            cat_to_exclude=exclude, return_dataframes=True)

In [ ]:
results_ex2.keys()

In [ ]:
occ_df = results_ex2['occupancy']

In [ ]:
occ_df.head()

In [ ]:
occ_df.tail()

In [ ]:
occ_df.info()

### Example 3 - Running via a Python script
Of course, you don't have to run Python statements through an IPython notebook. You can simply create a short Python script and run that directly in a terminal. An example, `test_shortstay.py`, can be found in the `scripts` subfolder of the hillmaker-examples project. Here's what it looks like - you can modify as necessary for your needs. There is another example in that folder as well, `test_obsim_log.py`, that is slightly more complex in that the input data has raw simulation times (i.e. minutes past t=0) and we need to do some datetime math to turn them into calendar based inputs.

In [ ]:
import pandas as pd

import hillmaker as hm

file_stopdata = '../data/ShortStay.csv'

# Required inputs
scenario = 'sstest_60'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start = '1/1/1996'
end = '3/30/1996 23:45'

# Optional inputs
tot_fld_name = 'SSU'
bin_mins = 60


df = pd.read_csv(file_stopdata, parse_dates=[in_fld_name, out_fld_name])

hm.make_hills(scenario, df, in_fld_name, out_fld_name,
                     start, end, cat_fld_name,
                     tot_fld_name, bin_mins,
                     cat_to_exclude=None,
                     verbose=1)

More elaborate versions of scripts like `test_shortstay.py` can be envisioned. For example, an entire folder of input data files could be processed by simple enclosing the `hm.make_hills` call inside a loop over the collection of input files:

In [ ]:
for log_fn in glob.glob('logs/*.csv'):
    
        # Read the log file and filter by included categories
        stops_df = pd.read_csv(log_fn, parse_dates=[in_fld_name, out_fld_name])
        
        hm.make_hills(scenario, df, in_fld_name, out_fld_name, start, end, cat_fld_name)
        ...